# Notebook: SBERT-First Classification with Enhanced Web Scraping


* **Author:** Carlos Garcia
* **Objective:** This notebook tests an advanced "SBERT-First" classification strategy. The goal is to improve upon the initial rule-based system by leveraging a Sentence Transformer model as the primary classifier, fed by text from an enhanced web scraping and web search pipeline.

### **Strategy Overview**

This notebook implements a multi-stage classification pipeline:
1.  **SBERT-First Approach:** For each Point of Interest (POI), we first attempt to classify it using a Sentence Transformer model (`all-MiniLM-L6-v2`).
2.  **Enhanced Text Retrieval:** To provide rich text to SBERT, we implement a new, more robust text retrieval system:
    * **Primary Scraping:** It first tries to scrape the website URL provided in the Overture Maps data. The scraper is enhanced with `trafilatura` for better content extraction and falls back to using the website's meta description if the main content is sparse.
    * **Web Search Fallback:** If the primary URL fails or yields no usable text, the system automatically performs a DuckDuckGo web search for the POI's name and attempts to scrape the top search results.
3.  **Rule-Based Fallback:** If the SBERT model cannot make a confident prediction (i.e., the similarity score is too low or no text could be found), the system falls back to the simpler rule-based keyword matching on the POI's name, as developed in the previous notebook.
4.  **Evaluation:** The final combined predictions are evaluated against the ground-truth categories to measure overall accuracy and the specific contribution of each method (SBERT vs. Rule-Based).

**Key Improvements Tested in This Notebook:**
* Using descriptive phrases for category embeddings (e.g., "a type of place known as coffee shop") to improve SBERT's semantic understanding.
* Using `trafilatura` and meta descriptions for more robust web scraping.
* Adding a DuckDuckGo search as a fallback to find websites for POIs that are missing them in the Overture data.

## Part 1: Setup and Configuration

This first section handles all necessary setup for the notebook. This includes:
* Importing all required Python libraries for data handling (`pandas`), AI/ML (`sentence-transformers`, `torch`, `sklearn`), and the new enhanced web scraping/search (`trafilatura`, `duckduckgo-search`).
* Setting key configuration variables that will be used throughout the notebook, such as the San Francisco bounding box and parameters for the SBERT model.


In [9]:
# --- Core and Data Handling Libraries ---
import pandas as pd
import numpy as np
import overturemaps
import re
import ast
from collections import defaultdict
import time
from urllib.parse import urljoin

# --- AI, ML, and Text Processing Libraries ---
# Make sure to install on you  environment:
# !pip install sentence-transformers scikit-learn
# !pip install trafilatura
# !pip install -U duckduckgo-search
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score, classification_report

# --- Web Scraping and Search Libraries ---
import requests
from bs4 import BeautifulSoup
import trafilatura
from duckduckgo_search import DDGS

# --- Display options for better notebook readability ---
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 150)

print("Libraries imported successfully.")

# --- Key Configuration Variables ---
# Bounding Box for San Francisco, CA
bbox_sf = (-122.5136, 37.7079, -122.3569, 37.8324)
print(f"Bounding box for San Francisco defined as: {bbox_sf}")

# SBERT Model Name and optimal similarity threshold
# This model is fast and provides good performance. The threshold was determined from prior tuning.
SBERT_MODEL_NAME = 'all-MiniLM-L6-v2'
SBERT_SIMILARITY_THRESHOLD = 0.25

Libraries imported successfully.
Bounding box for San Francisco defined as: (-122.5136, 37.7079, -122.3569, 37.8324)


## Part 2: Data Loading and Preprocessing

Here, we load our source data from Overture Maps. For this experimental run, we are loading the full dataset for San Francisco and then taking a **random sample of 100 POIs**. Working with a smaller sample allows for faster iteration and testing of our complex scraping and classification pipeline.

We also perform the same essential preprocessing steps as in our previous work:
* Extracting the `primary_name` and `primary_category` from the nested dictionary columns.
* Creating a `cleaned_name` by lowercasing and standardizing the POI name.
* Dropping any rows from our sample that are missing essential information required for classification and evaluation.

In [10]:
# --- Load and Preprocess Data ---
print("Fetching Overture Maps POI data for San Francisco...")
df_sf_pois = pd.DataFrame()
try:
    # We load the full dataset first, then take a sample for faster execution and testing.
    # For a full run, you would use df_sf_pois_full directly.
    place_data_batches = overturemaps.record_batch_reader("place", bbox=bbox_sf)
    df_sf_pois_full = place_data_batches.read_all().to_pandas()
    
    # Using a 100 POI sample for this demonstration run.
    df_sf_pois = df_sf_pois_full.sample(n=min(100, len(df_sf_pois_full)), random_state=42).copy()
    print(f"Successfully loaded and sampled {len(df_sf_pois)} POIs for San Francisco.")
except Exception as e:
    print(f"An error occurred during data loading: {e}")

if not df_sf_pois.empty:
    # --- Data Extraction from nested Overture columns ---
    df_sf_pois['primary_name'] = df_sf_pois['names'].apply(lambda x: x.get('primary') if isinstance(x, dict) else None)
    df_sf_pois['primary_category'] = df_sf_pois['categories'].apply(lambda x: x.get('primary') if isinstance(x, dict) else None)
    
    # --- Name Cleaning Function ---
    def clean_poi_name(name):
        if not isinstance(name, str): return ""
        name = name.lower()
        name = re.sub(r"[^a-z0-9\s'-]", '', name)
        name = name.strip()
        name = re.sub(r'\s+', ' ', name)
        return name
    df_sf_pois['cleaned_name'] = df_sf_pois['primary_name'].apply(clean_poi_name)

    # --- Drop Rows with Missing Essential Data for classification ---
    initial_rows = len(df_sf_pois)
    df_sf_pois.dropna(subset=['id', 'primary_name', 'primary_category'], inplace=True)
    df_sf_pois = df_sf_pois[df_sf_pois['cleaned_name'] != ""].copy()
    print(f"Dropped {initial_rows - len(df_sf_pois)} rows due to missing essential data.")
    print(f"Shape after preprocessing: {df_sf_pois.shape}")
    display(df_sf_pois.head())
else:
    print("DataFrame is empty. Skipping preprocessing.")

Fetching Overture Maps POI data for San Francisco...
Successfully loaded and sampled 100 POIs for San Francisco.
Dropped 0 rows due to missing essential data.
Shape after preprocessing: (100, 18)


,id,geometry,bbox,type,version,sources,names,categories,confidence,websites,socials,emails,phones,brand,addresses,primary_name,primary_category,cleaned_name
8904,08f2830829c5ad950320226c4002320e,b'\x00\x00\x00\x00\x01\xc0^\x9c\x11\xa4m23@B\xe3\xc9@PR(',"{'xmin': -122.43859100341797, 'xmax': -122.4385757446289, 'ymin': 37.779579162597656, 'ymax': 37.77958297729492}",place,0,"[{'property': '', 'dataset': 'meta', 'record_id': '750805774968210', 'update_time': '2025-02-24T08:00:00.000Z', 'confidence': 0.9793990828827596},...","{'primary': 'Primo Pizza', 'common': None, 'rules': None}","{'primary': 'pizza_restaurant', 'alternate': ['restaurant', 'italian_restaurant']}",0.995262,[http://www.primopizzasf.com],[https://www.facebook.com/750805774968210],None,[+14153599300],"{'wikidata': None, 'names': {'primary': 'Primo Pizza', 'common': None, 'rules': None}}","[{'freeform': '1064 Divisadero St', 'locality': 'San Francisco', 'postcode': '94115-4409', 'region': 'CA', 'country': 'US'}]",Primo Pizza,pizza_restaurant,primo pizza
17728,08f283082e320a2003b533874433800c,b'\x00\x00\x00\x00\x01\xc0^\x99\xa7p\x87Q\x98@B\xe2\x00H\xfb\x1b\x95',"{'xmin': -122.40084838867188, 'xmax': -122.40083312988281, 'ymin': 37.76563262939453, 'ymax': 37.7656364440918}",place,0,"[{'property': '', 'dataset': 'meta', 'record_id': '175907425952625', 'update_time': '2025-02-24T08:00:00.000Z', 'confidence': 0.93596425912137}]","{'primary': 'CULT Aimee Friberg', 'common': None, 'rules': None}","{'primary': 'art_gallery', 'alternate': ['arts_and_entertainment', 'topic_concert_venue']}",0.935964,[http://cultexhibitions.com/],[https://www.facebook.com/175907425952625],None,[+14152387385],None,"[{'freeform': '1401 16th St', 'locality': 'San Francisco', 'postcode': '94103-5109', 'region': 'CA', 'country': 'US'}]",CULT Aimee Friberg,art_gallery,cult aimee friberg
11775,08f283082c812baa03f71a5482b54811,b'\x00\x00\x00\x00\x01\xc0^\x9a\xc3Q\xf9\xc7\xd8@B\xdf*\xe6]\x97\x8b',"{'xmin': -122.41817474365234, 'xmax': -122.41815948486328, 'ymin': 37.74349594116211, 'ymax': 37.743499755859375}",place,0,"[{'property': '', 'dataset': 'meta', 'record_id': '164038230338841', 'update_time': '2025-02-24T08:00:00.000Z', 'confidence': 0.9793990828827596}]","{'primary': 'Esmeralda stairs', 'common': None, 'rules': None}","{'primary': 'park', 'alternate': ['playground', 'active_life']}",0.979399,[http://sfist.com/2015/06/30/the_steps_of_san_francisco_the_esme.php],[https://www.facebook.com/164038230338841],None,None,None,"[{'freeform': '101 Coleridge St', 'locality': 'San Francisco', 'postcode': '94110-5112', 'region': 'CA', 'country': 'US'}]",Esmeralda stairs,park,esmeralda stairs
13035,08f283082563170403ff2eb4b3e53cf0,b'\x00\x00\x00\x00\x01\xc0^\x98\xc4t8\xb7\x01@B\xdd\x14W\xa5\xd9C',"{'xmin': -122.38699340820312, 'xmax': -122.38697814941406, 'ymin': 37.72718048095703, 'ymax': 37.7271842956543}",place,0,"[{'property': '', 'dataset': 'meta', 'record_id': '144912772203952', 'update_time': '2025-02-24T08:00:00.000Z', 'confidence': 0.8941256830601093},...","{'primary': 'Excelsior Roofing Co.', 'common': None, 'rules': None}","{'primary': 'roofing', 'alternate': ['contractor', 'professional_services']}",0.975649,[http://www.excelsiorroofing.com/],[https://www.facebook.com/144912772203952],None,[+14158224488],None,"[{'freeform': '1340 Underwood Ave', 'locality': 'San Francisco', 'postcode': '94124-3309', 'region': 'CA', 'country': 'US'}]",Excelsior Roofing Co.,roofing,excelsior roofing co
28947,08f283082a33503003f7b0c2d4b60ce2,b'\x00\x00\x00\x00\x01\xc0^\x99\xaa\xf44\xf0\xa3@B\xe4\xdc\xc2\rV*',"{'xmin': -122.40106201171875, 'xmax': -122.40104675292969, 'ymin': 37.787986755371094, 'ymax': 37.78799057006836}",place,0,"[{'property': '', 'dataset': 'meta', 'record_id': '281811015494371', 'update_time': '2025-02-24T08:00:00.000Z', 'confidence': 0.865}]","{'primary': 'Dr. Steven John Adams D.C.', 'common': None, 'rules': None}","{'primary': 'chiropractor', 'alternate': ['health_and_medical', 'family_prac

## Part 3: Define Helper Functions for the Pipeline

This section centralizes all the functions needed for our pipeline. Consolidating function definitions here makes the main execution logic in Part 5 cleaner and easier to read. These functions handle scraping, searching, SBERT classification, and the rule-based fallback.

In [11]:
# --- Define All Helper Functions ---

# 1. Scraping Utilities
def get_meta_description(html_content):
    if not html_content: return ""
    soup = BeautifulSoup(html_content, 'html.parser')
    meta_tag = soup.find('meta', attrs={'name': 'description'})
    return meta_tag['content'].strip() if meta_tag and meta_tag.get('content') else ""

def extract_main_text(url, min_len=100):
    """Enhanced text extractor using trafilatura with a meta description fallback."""
    if not url or not isinstance(url, str): return ""
    if not url.startswith(('http://', 'https://')): url = 'http://' + url
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'} 
        downloaded = requests.get(url, timeout=7, allow_redirects=True, headers=headers)
        downloaded.raise_for_status()
        
        extracted_text = trafilatura.extract(downloaded.text, include_comments=False, include_tables=False, no_fallback=True) 
        if extracted_text and len(extracted_text) >= min_len:
            return extracted_text.lower()[:4000] # Truncate to 4000 chars to manage embedding size

        meta_desc = get_meta_description(downloaded.text)
        if meta_desc and len(meta_desc) >= 40:
            print(f"        -> Fallback success: Using meta description (length {len(meta_desc)}) from {url}")
            return meta_desc.lower()
            
        return "" 
    except (requests.RequestException, Exception):
        return ""

# 2. Search and Scrape Orchestrator
def search_and_scrape(poi_name, max_search_results=3):
    """Performs a web search for the POI name and tries to scrape the top results."""
    if not poi_name: return None
    print(f"    -> Performing web search for: '{poi_name}' using DuckDuckGo")
    query = f"{poi_name} San Francisco official website"
    try:
        with DDGS() as ddgs:
            search_results = list(ddgs.text(query, max_results=max_search_results))
        if not search_results: return None
        search_result_urls = [r['href'] for r in search_results]
    except Exception as e:
        print(f"      -> DuckDuckGo search failed: {e}")
        return None

    skip_domains = ['facebook.com', 'twitter.com', 'yelp.com', 'instagram.com', 'linkedin.com', 'mapquest.com', 'yellowpages.com']
    for url in search_result_urls:
        if any(skip_domain in url.lower() for skip_domain in skip_domains): continue
        text = extract_main_text(url) 
        if text and len(text) >= 50:
            print(f"        -> Web search success: Scraped {len(text)} chars from {url}")
            return text
    return None

# 3. SBERT Classification Function
def classify_text_with_sbert_flat(text_to_classify, sbert_model, embeddings, categories, threshold):
    """Classifies text against pre-computed descriptive category embeddings."""
    if not text_to_classify or not isinstance(text_to_classify, str) or len(text_to_classify.strip()) < 50:
        return "other_sbert_input_too_short"
    
    text_embedding = sbert_model.encode(text_to_classify, convert_to_tensor=True, show_progress_bar=False)
    cosine_scores = util.cos_sim(text_embedding, embeddings)
    highest_score = cosine_scores[0].max().item()
    best_match_index = cosine_scores[0].argmax().item()
    
    return categories[best_match_index] if highest_score >= threshold else "other_sbert_low_similarity"

# 4. Rule-Based Classification Function
def classify_poi_by_name_substring(cleaned_name, rules_dict, sorted_keywords):
    """Classifies a POI name using a dictionary of substring rules."""
    if not isinstance(cleaned_name, str) or not cleaned_name: return 'other'
    for keyword in sorted_keywords:
        if keyword in cleaned_name: return rules_dict[keyword]
    return 'other'

print("All helper functions for pipeline are defined.")

All helper functions for pipeline are defined.


## Part 4: Pre-computation of Classification Assets

This section prepares the assets needed by our classifiers. These steps are computationally intensive and only need to be run once per session before the main pipeline execution.

1.  **SBERT Setup:** We load the `all-MiniLM-L6-v2` Sentence Transformer model. Crucially, instead of just embedding the raw category codes (e.g., `coffee_shop`), we create more **descriptive phrases** (e.g., "a type of place or service known as coffee shop"). This provides richer semantic context to the model, helping it make more accurate similarity comparisons. These descriptive phrases are then encoded into embeddings.
2.  **Rule-Based System Setup:** We prepare the keyword dictionary and the sorted list of keywords for our rule-based fallback classifier, which will be used if the SBERT-first approach fails.

In [13]:
# --- 1. Setup SBERT Model and Generate Category Embeddings ---
print("--- Setting up SBERT Classifier with Descriptive Phrases ---")
sbert_model = SentenceTransformer(SBERT_MODEL_NAME)
print(f"SBERT model '{SBERT_MODEL_NAME}' loaded.")

df_taxonomy = pd.read_csv('overture_categories.csv', delimiter=';')
overture_category_names_flat = [str(name) for name in df_taxonomy['Category code'].tolist() if pd.notna(name)]
descriptive_category_phrases = [f"a type of place or service known as {code.replace('_', ' ')}" for code in overture_category_names_flat]
print(f"Generating embeddings for {len(descriptive_category_phrases)} descriptive phrases...")
category_embeddings_flat = sbert_model.encode(descriptive_category_phrases, convert_to_tensor=True, show_progress_bar=True)
print("Descriptive category embeddings generated and ready.")

# --- 2. Setup Rule-Based Fallback System ---
print("\n--- Setting up Rule-Based Fallback System ---")
keyword_to_category_rules = {}
for category_code in overture_category_names_flat:
    keyword_to_category_rules[category_code.replace('_', ' ').strip()] = category_code

additional_rules = {'atm': 'atms', 'bank': 'bank_credit_union', 'gym': 'gym', 'fitness': 'gym', 'pharmacy': 'pharmacy', 'drugstore': 'pharmacy', 'bar': 'bar', 'pub': 'pub', 'clinic': 'clinic', 'cafe': 'cafe'}
keyword_to_category_rules.update(additional_rules)
specific_rules = {"summer camp": "educational_camp", "dog walking": "dog_walkers", "recology": "waste_management_services", "pizza": "pizza_restaurant"}
keyword_to_category_rules.update(specific_rules)
print(f"Total rules created: {len(keyword_to_category_rules)}")

sorted_keywords_for_fallback = tuple(sorted(keyword_to_category_rules.keys(), key=len, reverse=True))
print(f"Prepared {len(sorted_keywords_for_fallback)} sorted keywords for fallback.")

--- Setting up SBERT Classifier with Descriptive Phrases ---
SBERT model 'all-MiniLM-L6-v2' loaded.
Generating embeddings for 2117 descriptive phrases...


Batches:   0%|          | 0/67 [00:00<?, ?it/s]

Descriptive category embeddings generated and ready.

--- Setting up Rule-Based Fallback System ---
Total rules created: 2125
Prepared 2125 sorted keywords for fallback.


## Part 5: Execute the SBERT-First Classification Pipeline

This is the main execution block of the notebook. It iterates through each of the 100 POIs in our sample DataFrame and applies our full, multi-stage classification logic to generate a `final_pred_category`.

The logic for each POI is as follows:
1.  **Try SBERT First:**
    * Attempt to scrape text from the Overture-provided website using the enhanced scraper (`get_website_text_for_poi`).
    * If that fails, fall back to a web search for the POI name using DuckDuckGo (`search_and_scrape`).
    * If any usable text is found from either source, classify it with the SBERT model (`classify_text_with_sbert_flat`).
    * If the prediction is confident (i.e., above the similarity threshold), we use it as the `final_prediction` and note the source as "SBERT".
2.  **Rule-Based Fallback:**
    * If the SBERT approach did not produce a confident prediction (e.g., no text was found, or similarity was too low), we then fall back to the simple rule-based classifier (`classify_poi_by_name_substring`) on the POI's `cleaned_name`.
    * If the rule-based system finds a match (not "other"), we use that as the `final_prediction` and note the source as "Rule-Based Fallback".
3.  **Final Outcome:**
    * If neither stage produces a specific category, the `final_prediction` remains "other".
    * The final prediction and its source (`SBERT`, `Rule-Based Fallback`, or `Fallback_Other`) are stored in new columns in the DataFrame.

In [14]:
# --- Execute the SBERT-First Pipeline ---
print("--- Applying SBERT-First Pipeline (with Web Search Fallback) ---")
df_sf_pois['final_pred_category'] = pd.NA
df_sf_pois['prediction_source'] = pd.NA

# This loop applies the full logic to each row of the DataFrame
for index, row in df_sf_pois.iterrows():
    print(f"Processing POI: {row['primary_name']}")
    final_prediction = 'other'
    prediction_source = 'Fallback_Other'
    
    # --- SBERT-First Logic ---
    text_for_sbert = get_website_text_for_poi(row['websites'])
    if not text_for_sbert:
        # If Overture URL fails, try searching the web
        text_for_sbert = search_and_scrape(row['primary_name'])
    
    if text_for_sbert:
        sbert_pred = classify_text_with_sbert_flat(text_for_sbert, sbert_model, category_embeddings_flat, overture_category_names_flat, SBERT_SIMILARITY_THRESHOLD)
        if not sbert_pred.startswith('other_sbert'):
            final_prediction = sbert_pred
            prediction_source = 'SBERT'
            print(f"  --> SBERT Prediction: {final_prediction}")
    
    # --- Rule-Based Fallback Logic ---
    if final_prediction == 'other':
        rule_pred = classify_poi_by_name_substring(row['cleaned_name'], keyword_to_category_rules, sorted_keywords_for_fallback)
        if rule_pred != 'other':
            final_prediction = rule_pred
            prediction_source = 'Rule-Based Fallback'
            print(f"  --> Rule-Based Fallback Prediction: {final_prediction}")

    df_sf_pois.loc[index, 'final_pred_category'] = final_prediction
    df_sf_pois.loc[index, 'prediction_source'] = prediction_source

print("\n--- Pipeline execution complete. ---")
display(df_sf_pois[['primary_name', 'primary_category', 'final_pred_category', 'prediction_source']].head(10))

--- Applying SBERT-First Pipeline (with Web Search Fallback) ---
Processing POI: Primo Pizza
  --> SBERT Prediction: pizza_restaurant
Processing POI: CULT Aimee Friberg
  --> SBERT Prediction: art_gallery
Processing POI: Esmeralda stairs
Processing POI: Excelsior Roofing Co.
  --> SBERT Prediction: roofing
Processing POI: Dr. Steven John Adams D.C.
  --> SBERT Prediction: sports_medicine
Processing POI: Faith Temple Cogic
  --> SBERT Prediction: jehovahs_witness_kingdom_hall
Processing POI: Pho Huynh Sang
    -> Performing web search for: 'Pho Huynh Sang' using DuckDuckGo
Processing POI: 1886 Square-Rigger Balclutha
  --> SBERT Prediction: tubing_provider
Processing POI: Lakeview
    -> Performing web search for: 'Lakeview' using DuckDuckGo
        -> Web search success: Scraped 3379 chars from http://www.sfrecpark.org/
  --> SBERT Prediction: country_dance_hall
Processing POI: Kai Aknin
    -> Performing web search for: 'Kai Aknin' using DuckDuckGo
Processing POI: 十九街華人浸信會 (Nacbc)
  -

,primary_name,primary_category,final_pred_category,prediction_source
8904,Primo Pizza,pizza_restaurant,pizza_restaurant,SBERT
17728,CULT Aimee Friberg,art_gallery,art_gallery,SBERT
11775,Esmeralda stairs,park,other,Fallback_Other
13035,Excelsior Roofing Co.,roofing,roofing,SBERT
28947,Dr. Steven John Adams D.C.,chiropractor,sports_medicine,SBERT
12403,Faith Temple Cogic,church_cathedral,jehovahs_witness_kingdom_hall,SBERT
7950,Pho Huynh Sang,vietnamese_restaurant,other,Fallback_Other
35389,1886 Square-Rigger Balclutha,monument,tubing_provider,SBERT
1670,Lakeview,active_life,country_dance_hall,SBERT
31780,Kai Aknin,financial_service,other,Fallback_Other


## Part 6: Final Evaluation

The final step is to evaluate the performance of our combined, multi-stage pipeline on the 100-POI sample. We will analyze:
1.  **Overall Accuracy:** The accuracy of our `final_pred_category` against the ground truth.
2.  **Accuracy of Non-"other" Predictions:** The accuracy for only the POIs where our pipeline was able to assign a specific category.
3.  **Contribution Analysis:** We will break down the accuracy by the source of the prediction (`SBERT` vs. `Rule-Based Fallback`) to understand which part of the pipeline is performing better and on how many POIs.
4.  **Detailed Report:** A classification report showing precision, recall, and F1-score for each category.

In [15]:
# --- Final Evaluation ---
print("--- Final Evaluation of SBERT-First Pipeline ---")

if 'final_pred_category' in df_sf_pois.columns:
    # 1. Overall Accuracy
    overall_accuracy = accuracy_score(df_sf_pois['primary_category'], df_sf_pois['final_pred_category'])
    print(f"Overall Accuracy (SBERT-First w/ Fallback): {overall_accuracy:.2%}")

    # 2. Accuracy on non-"other" predictions
    df_classified_final = df_sf_pois[df_sf_pois['final_pred_category'] != 'other']
    if not df_classified_final.empty:
        accuracy_non_other = accuracy_score(df_classified_final['primary_category'], df_classified_final['final_pred_category'])
        print(f"Accuracy for Non-'other' Predictions: {accuracy_non_other:.2%}")
        print(f"(Calculated on {len(df_classified_final)} POIs where a specific category was assigned)")
    else:
        print("No POIs were classified with a specific category.")
    print("-" * 70)

    # 3. Analyze contribution of each prediction source
    print("Prediction Source Distribution:")
    display(df_sf_pois['prediction_source'].value_counts())
    print("-" * 70)
    
    print("Accuracy by Prediction Source:")
    for source in ["SBERT", "Rule-Based Fallback"]:
        df_source = df_sf_pois[df_sf_pois['prediction_source'] == source]
        if not df_source.empty:
            accuracy_source = accuracy_score(df_source['primary_category'], df_source['final_pred_category'])
            print(f"  Accuracy of '{source}' specific predictions: {accuracy_source:.2%} (on {len(df_source)} POIs)")
else:
    print("Final evaluation could not be completed.")

# 4. Detailed Classification Report for Non-'other' predictions
if not df_classified_final.empty:
    print("\n--- Detailed Classification Report (for Non-'other' Predictions) ---")
    report = classification_report(df_classified_final['primary_category'], df_classified_final['final_pred_category'], zero_division=0)
    print(report)
else:
    print("\nNo data for detailed classification report.")

--- Final Evaluation of SBERT-First Pipeline ---
Overall Accuracy (SBERT-First w/ Fallback): 21.00%
Accuracy for Non-'other' Predictions: 22.58%
(Calculated on 93 POIs where a specific category was assigned)
----------------------------------------------------------------------
Prediction Source Distribution:


prediction_source
SBERT                  90
Fallback_Other          7
Rule-Based Fallback     3
Name: count, dtype: int64

----------------------------------------------------------------------
Accuracy by Prediction Source:
  Accuracy of 'SBERT' specific predictions: 21.11% (on 90 POIs)
  Accuracy of 'Rule-Based Fallback' specific predictions: 66.67% (on 3 POIs)

--- Detailed Classification Report (for Non-'other' Predictions) ---
                                     precision    recall  f1-score   support

                      accommodation       0.00      0.00      0.00         1
                        active_life       0.00      0.00      0.00         1
    addiction_rehabilitation_center       0.00      0.00      0.00         0
                american_restaurant       1.00      1.00      1.00         1
             architectural_designer       0.00      0.00      0.00         1
                        art_gallery       1.00      0.50      0.67         2
                               atms       1.00      1.00      1.00         2
                             bakery       0.00      0.00      0.00    

## Summary of Findings & Next Steps

This notebook demonstrated a significantly enhanced "SBERT-First" pipeline.

### Key Findings:
* **Enhanced Scraping is Effective:** The combination of `trafilatura`, meta description fallback, and a DuckDuckGo search fallback allowed us to find and process text for POIs that failed in the previous, simpler scraping attempt.
* **Descriptive Embeddings Show Promise:** Using descriptive phrases for category embeddings (e.g., "a type of place known as...") provides richer context for the SBERT model.
* **SBERT-First Performance:** In our 100-POI sample, the SBERT model was able to provide a classification for a majority of the items, with the rule-based system acting as a fallback. The final evaluation metrics show the combined accuracy of this approach. *[Here you would add the final accuracy numbers from your evaluation cell, e.g., "The SBERT-first predictions had an accuracy of X%, and the overall combined accuracy was Y%."]*

### Next Steps for the Project:

1.  **Tune SBERT Threshold:** The `SBERT_SIMILARITY_THRESHOLD` (set to 0.25) can be tuned. A higher value would make the SBERT predictions more precise (fewer false positives) but might result in more POIs needing the rule-based fallback (lower recall).
2.  **Scale Up Processing:** Run this pipeline on a larger sample (e.g., 500 or 1000 POIs) to get more statistically significant performance metrics.
3.  **Error Analysis:** Perform a deeper analysis of the final mismatches.
    * *SBERT errors:* Why did SBERT choose the wrong category? Was the scraped text misleading? Was the true category semantically ambiguous?
    * *Rule-based errors:* Are there simple new keyword rules that could be added to fix common fallback errors?
4.  **Complete KR 2.2:** Build the manually-labeled ~200 POI evaluation set. This "golden dataset" is essential for objectively measuring performance against your OKRs.
5.  **Evaluate Other Models (KR 2.1):** Compare this SBERT approach against another distinct ML approach (e.g., a traditional TF-IDF + Logistic Regression model on the same scraped text) using the new golden evaluation set.